<a href="https://colab.research.google.com/github/abidgymnastiar/JobsheetMechineLearning/blob/main/jobsheet_4/tugas_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# Import required libraries
from pathlib import Path
import matplotlib.image as mpimg
import cv2
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score

# Directory untuk data training dan testing
train_dir = "/content/drive/MyDrive/MechineLearning/images/training"
test_dir = "/content/drive/MyDrive/MechineLearning/images/test"

# Fungsi untuk memuat dataset
def load_dataset(img_dir):
    p = Path(img_dir)
    dirs = p.glob('*')

    img_list = []

    for dir in dirs:
        label = str(dir).split('/')[-1]  # Ambil nama folder sebagai label
        for file in dir.glob('*.jpg'):
            img = mpimg.imread(file)

            if img is not None:
                img_list.append((img, label))

    return img_list

# Fungsi untuk menstandarkan ukuran gambar
def standarized_input(image):
    std_img = cv2.resize(image, (1100, 600))
    return std_img

# Fungsi untuk encoding label
def label_encoder(label):
    return 1 if label == 'day' else 0

# Fungsi untuk melakukan pra-pengolahan
def preprocess(img_list):
    std_img_list = []
    for item in img_list:
        image = item[0]
        label = item[1]

        # Standarized the image
        std_img = standarized_input(image)

        # Create the label
        img_label = label_encoder(label)

        std_img_list.append((std_img, img_label))

    return std_img_list

# Fungsi untuk menghitung histogram gambar
def calculate_histogram(image):
    img_hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)  # Convert to HSV
    histogram = cv2.calcHist([img_hsv], [2], None, [256], [0, 256])  # Hitung histogram
    histogram = cv2.normalize(histogram, histogram).flatten()  # Normalisasi histogram
    return histogram

# Fungsi untuk memproses dataset dan menghitung histogram
def preprocess_with_histogram(img_list):
    features = []
    labels = []

    for item in img_list:
        image = item[0]
        label = item[1]

        # Hitung histogram
        hist = calculate_histogram(image)

        features.append(hist)
        labels.append(label)

    return np.array(features), np.array(labels)

# Load dan proses data training
train_img = load_dataset(train_dir)
train_std_img_list = preprocess(train_img)
X_train, y_train = preprocess_with_histogram(train_std_img_list)

# Membangun model SVM
model = svm.SVC(kernel='linear')  # Menggunakan kernel linear
model.fit(X_train, y_train)  # Melatih model

# Menghitung akurasi data training
train_predictions = model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_predictions)

# Load dan proses data pengujian
test_img = load_dataset(test_dir)
test_std_img_list = preprocess(test_img)
X_test, y_test = preprocess_with_histogram(test_std_img_list)

# Prediksi menggunakan model untuk data testing
y_pred = model.predict(X_test)

# Menghitung akurasi data testing
test_accuracy = accuracy_score(y_test, y_pred)

# Menampilkan hasil akurasi
print(f"Akurasi Data Training: {train_accuracy:.4f}")
print(f"Akurasi Data Testing: {test_accuracy:.4f}")


Akurasi Data Training: 0.9958
Akurasi Data Testing: 0.9563
